In [ ]:
from __future__ import division
import mpl_toolkits
from cmcrameri import cm
mpl_toolkits.__path__.append('/usr/lib/python3.6/dist-packages/mpl_toolkits/')
from mpl_toolkits.basemap import Basemap
import matplotlib.ticker as ticker
import h5py
import numpy as np
import matplotlib.pyplot as plt
import xlrd
from matplotlib import colors as c
import matplotlib.ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable


class FormatScalarFormatter(matplotlib.ticker.ScalarFormatter):
    def __init__(self, fformat="%1.1f", offset=True, mathText=True):
        self.fformat = fformat
        matplotlib.ticker.ScalarFormatter.__init__(self,useOffset=offset,
                                                        useMathText=mathText)
    def _set_format(self, vmin, vmax):
        self.format = self.fformat
        if self._useMathText:
            self.format = '$%s$' % matplotlib.ticker._mathdefault(self.format)
lat = np.arange(-89.975, 90, 0.05)
lon = np.arange(-179.975, 180, 0.05)

file = xlrd.open_workbook(filename)
sheet = file.sheet_by_index(1)

date = []
lats = []
lons = []
LstMax = []

for i in range(1, 6372):
    lats.append(sheet.cell_value(i, 2))
    lons.append(sheet.cell_value(i, 3))
    LstMax.append(sheet.cell_value(i, 1))

LstMax = np.array(LstMax)
lats = np.array(lats)
lons = np.array(lons)

inds = LstMax.argsort()
LstMax_sorted = LstMax[inds]
lats_sorted = lats[inds]
lons_sorted = lons[inds]

fig = plt.gcf()
fig.set_size_inches(13, 6.5)
font_size = 30

# Create map: Locations of Each Year Maximum Land Surface Temperature
m = Basemap(projection='robin', lon_0 = 0)
m.shadedrelief()

m.drawcoastlines(color='grey', linewidth=0.2)
m.drawparallels(np.arange(-90.,120.,30.), labels=[1, 0, 0, 0],linewidth=0.3 ,dashes=[1, 4], fontsize= font_size)
m.drawmeridians(np.arange(0.,360.,60.),linewidth=0.3, dashes=[1, 4], fontsize = font_size )
clevs = np.linspace(np.nanmin(LstMax_sorted), np.nanmax(LstMax_sorted), 7, endpoint=True)
x, y = m(lons_sorted, lats_sorted)
cm1 = plt.scatter(x, y, c=LstMax_sorted, cmap = cm.acton_r, marker='.', edgecolors='k', linewidths=0.6)
def myfmt(x, pos):
    return '{0:.1f}'.format(x)
fmt = FormatScalarFormatter("%.1f")
cb1 = plt.colorbar(cm1, orientation="horizontal", ticks=clevs, pad=0.02, aspect=50, extendfrac='5%', fraction=0.038,format=ticker.FuncFormatter(myfmt))

cb1.ax.tick_params(labelsize = font_size)
cb1.set_label(label='a label',weight='bold')
cb1.set_label('Daily maximum diurnal temperature variations (°C)',fontsize = font_size)
plt.tight_layout()
plt.show()

